## Oracle 11g character set 한글 인코딩 문제

### Sample 코드

```python
from pydatafabric.vault_utils import get_secrets
oracle_info = get_secrets(mount_point="datafabric", path="oracle/entsal/prd")

from pydatafabric.ye import get_spark
spark = get_spark(extra_jars="gs://emart-datafabric-resources/jars/ojdbc8.jar,gs://emart-datafabric-resources/jars/orai18n.jar")
spark.conf.set("spark.sql.debug.maxToStringFields", 2000)

from functools import reduce

# AMERICAN_AMERICA.US7ASCII
# KOREAN_KOREA.KO16MSWIN949
# KOREAN_KOREA.KO16KSC5601
# KOREAN_KOREA.AL32UTF8
    
def read_oracle_to_df(spark, dbtable):
    df = spark.read.format("jdbc") \
        .option("url", oracle_info['jdbc_url']) \
        .option("user", oracle_info['user']) \
        .option("password", oracle_info['password']) \
        .option("driver", "oracle.jdbc.driver.OracleDriver") \
        .option("oracle.jdbc.timezoneAsRegion", "false") \
        .option("dbtable", dbtable) \
        .load()
    return df

def convert_col(col):
    return "".join([c.lower() if c.isupper() else c for c in col])

df = read_oracle_to_df(spark, "(SELECT UTL_RAW.CAST_TO_RAW(CONVERT(COMMENTS, 'AL32UTF8', 'KO16KSC5601')) AS COMMENTS FROM ALL_COL_COMMENTS WHERE TABLE_NAME = 'GTPSS_MALL_SALE') INPUT")


from pyspark.sql.functions import *

# DataFrame의 스키마 이름 가져옴
cols = df.schema.names

# DataFrame Column 이름을 소문자로 변환
df = reduce(lambda _df, col: _df.withColumnRenamed(col, convert_col(col)), [df, *cols])
df.printSchema()

df.show(n=1, truncate=False, vertical=False)

from pydatafabric.gcp import df_to_bq_table, bq_insert_overwrite

gcp_table = "us7ascii_to_utf"
df_to_bq_table(df, "temp_1d", f"{gcp_table}", project="emart-datafabric", mode="overwrite")    

from pydatafabric.gcp import get_bigquery_client

bq = get_bigquery_client(project="emart-datafabric") # 프로젝트 꼭 지정
r = bq.query(f"SELECT SAFE_CONVERT_BYTES_TO_STRING(comments) FROM temp_1d.{gcp_table} LIMIT 10")

df = r.result().to_dataframe()

spark.stop()

```